In [2]:
!mkdir -p  ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [3]:
!kaggle datasets download -d salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
 99% 1.06G/1.06G [00:04<00:00, 163MB/s]
100% 1.06G/1.06G [00:05<00:00, 226MB/s]


In [5]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip')
zip_ref.extractall('/content')
zip_ref.close()

In [22]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout,Flatten
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator,img_to_array,load_img,array_to_img
from keras.applications.vgg16 import VGG16


In [14]:
#Data Augmentation

train_Aug = ImageDataGenerator(
    rescale = 1./255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range= 0.2,
    rotation_range = 0.2,
    horizontal_flip=True,
    zoom_range = 0.2
)

In [25]:
test_Aug = ImageDataGenerator(
    rescale = 1./255,

)


In [26]:
train_aug_data = train_Aug.flow_from_directory(
    '/content/train',
    target_size=(150,150),
    class_mode = 'binary',
    batch_size = 32

)

Found 20000 images belonging to 2 classes.


In [27]:
test_aug_data = test_Aug.flow_from_directory(
    '/content/test',
    target_size = (150,150),
    class_mode = 'binary',
    batch_size = 32
)

Found 5000 images belonging to 2 classes.


In [28]:
conv_base = VGG16(
    weights = 'imagenet',
    include_top = False,
    input_shape = (150,150,3)

)

In [30]:
conv_base.trainable = True
set_trainable = False

In [33]:
'''for layer in conv_base.layers:
  if layer.name == 'block5_conv1':
    set_trainable = True
  if set_trainable:
    layer.trainable = True

  else:
    layer.trainable = False'''


In [37]:
for layer in conv_base.layers:
  if layer.name == 'block5_conv1':
    layers.trainable = True
  else:
    layers.trainable = False

In [38]:
for layer in conv_base.layers:
  print(layer.name,   layer.trainable)

input_2 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 True
block5_conv2 True
block5_conv3 True
block5_pool True


In [40]:
model = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [42]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 8192)              0         
                                                                 
 dense_1 (Dense)             (None, 256)               2097408   
                                                                 
 dense_2 (Dense)             (None, 1)                 257       
                                                                 
Total params: 16812353 (64.13 MB)
Trainable params: 9177089 (35.01 MB)
Non-trainable params: 7635264 (29.13 MB)
_________________________________________________________________


In [50]:
model.compile(optimizer = keras.optimizers.RMSprop(lr = 1e-5),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:
history = model.fit_generator(train_aug_data,epochs = 5, validation_data= test_aug_data)

<ipython-input-52-aa5cbefa9930>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_aug_data,epochs = 5, validation_data= test_aug_data)


Epoch 1/5
213/625 [=========>....................] - ETA: 1:11:43 - loss: 1.4369 - accuracy: 0.4938